In [5]:
import disclosuregame
from disclosuregame.Agents.bayes import BayesianSignaller, Responder
from disclosuregame.Agents.cpt import ProspectTheorySignaller
from disclosuregame.Agents.heuristic import LexicographicSignaller
from disclosuregame.Agents.payoff import BayesianPayoffSignaller
from disclosuregame.Games.game import SimpleGame

In [16]:
response_alpha=[[1.0, 1.0], [1.0, 1.0], [1.0, 1.0]]
type_alpha=[1.0, 1.0, 1.0]

def make_agents(agent_fn):
    a = agent_fn(player_type=0)
    b = Responder(player_type=1)
    game = SimpleGame(baby_payoff=10,referral_cost=9)
    a.init_payoffs(game.woman_baby_payoff, game.woman_social_payoff, response_weights=response_alpha, type_weights=type_alpha)
    return a, b

def play_games(agent, opponent):
    a.log_signal(0)
    a.update_counts(0, b, 10)
    a.log_signal(0)
    a.update_counts(0, b, 10)
    a.log_signal(1)
    a.update_counts(0, b, 9)

## Lexicographic

## Payoff Based Bayesian Risk Minimisation

[(l,10),(m,10),(h,10),(m,9),(h,9),(h,8)], with αi = 1 for all i. 
After playing the three rounds, nl,10 = 2, and nm,9 = 1.

In [20]:
a, b = make_agents(BayesianPayoffSignaller)
# Reset the payoff count, because init_payoffs counts 
# based on 'possible experiences' not literal alpha
for signal, payoffs in a.payoff_count.iteritems():
    for payoff, val in payoffs.iteritems():
        a.payoff_count[signal][payoff] = 1
play_games(a, b)
a.update_beliefs()
print a.payoff_count
print a.risk(0)
print a.risk(1)
print a.risk(2)

{0: {10: 3.0}, 1: {9: 2.0, 10: 1}, 2: {8: 1, 9: 1, 10: 1}}
-10.0
-9.33333333333
-9.0


## Bayesian Risk Minimisation

In [26]:
a, b = make_agents(BayesianSignaller)

In [27]:
print a.signal_matches, a.type_matches
play_games(a, b)
print a.signal_matches, a.type_matches
a.update_beliefs()

{0: 0.0, 1: 0.0, 2: 0.0} {0: 0.0, 1: 0.0, 2: 0.0}
{0: 2.0, 1: 1.0, 2: 0.0} {0: 0.0, 1: 3.0, 2: 0.0}


In [9]:
a.risk(0)

-9.999999999999998

In [10]:
a.risk(1)

-9.333333333333332

In [11]:
a.risk(2)

-8.5

## CPT Example

In [14]:
a, b = make_agents(ProspectTheorySignaller)
play_games(a, b)
a.update_beliefs()

In [20]:
prospects = a.collect_prospects(0)
print prospects
a.cpt_value(prospects)

[(10, 0.5), (10, 0.16666666666666666), (10, 0.125), (10, 0.125), (10, 0.041666666666666664), (10, 0.041666666666666664)]


7.585775750291834

In [17]:
prospects = a.collect_prospects(1)
print prospects
a.cpt_value(prospects)

[(10, 0.1111111111111111), (10, 0.1111111111111111), (10, 0.05555555555555555), (10, 0.05555555555555555), (9, 0.4444444444444444), (9, 0.2222222222222222)]


7.139727151083673

In [18]:
prospects = a.collect_prospects(2)
print prospects
a.cpt_value(prospects)

[(10, 0.08333333333333333), (10, 0.08333333333333333), (9, 0.08333333333333333), (9, 0.08333333333333333), (8, 0.3333333333333333), (8, 0.3333333333333333)]


6.622393760235688